This notebook is used to train the primary gait transformer, which translates a buffer of kinematics data to a gait state. For newcomers to this repo, this is where you should start if you want to train your own Transformer.

In [ ]:
# import tensorflow as tf
# tf.config.list_physical_devices('GPU')
# tf.test.is_built_with_cuda()
import os, sys
sys.path.append('../')
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import pyarrow.parquet as pq
import time, datetime
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from datasets import WindowedGaitDataset, ToTensor, ExobootDataset
from gait_transformer import GaitTransformer
from save_best_model import SaveBestModel
from training_utils import phase_dist, unscale_kinematics, unscale_gait_state
from torch_training_utils import GaitLoss, EWC, enum_parameters


import random
# Set the seed value all over the place to make this reproducible.
seed_val = 42


random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

#RUN THIS ON COLAB
ON_COLAB = False
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/Phase ML Data/'

Import the datafiles containing the walking data used to train the network

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/r01_ordered_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/r01_randomized_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/dataport_ordered_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/dataport_randomized_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/gt_ordered_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/gt_randomized_corrupt_time.csv .

In [ ]:
!gsutil cp -r gs://ml_gait_estimation/r01_ordered_stairs_label_corrupt_time.csv .
!gsutil cp -r gs://ml_gait_estimation/r01_randomized_stairs_label_corrupt_time.csv .
!gsutil cp -r gs://ml_gait_estimation/dataport_ordered_stairs_label_corrupt_time.csv .
!gsutil cp -r gs://ml_gait_estimation/dataport_randomized_stairs_label_corrupt_time.csv .
!gsutil cp -r gs://ml_gait_estimation/gt_ordered_stairs_label_corrupt_time.csv .
!gsutil cp -r gs://ml_gait_estimation/gt_randomized_stairs_label_corrupt_time.csv .

Set up dataframes, Datasets, and DataLoaders to contain the kinematics and gait state labels

In [ ]:
window_size = 150 #set the number of kinematics in the buffer to the transformer

#set up kinematics scaling
meas_scale = np.array([[-69.35951035,  27.62815047],\
                        [-456.18013759,  401.13782617],\
                        [-63.71649984,  22.06632622],\
                        [-213.4786175,   396.93801619],\
                        [-35.26603985,  20.78473636],\
                        [-20.95456523,  14.63961137],\
                          [0,1]])

#set up gait state scaling
speed_scale = (0,2)
incline_scale = (-10,10)
stair_height_scale = (-1,1)

#set up filenames for each data file
filename_dataport_ordered = 'dataport_ordered_stairs_label_corrupt_time.csv'
filename_dataport_randomized = 'dataport_randomized_stairs_label_corrupt_time.csv'

filename_r01_ordered = 'r01_ordered_stairs_label_corrupt_time.csv'
filename_r01_randomized = 'r01_randomized_stairs_label_corrupt_time.csv'

filename_gt_ordered = 'gt_ordered_stairs_label_corrupt_time.csv'
filename_gt_randomized = 'gt_randomized_stairs_label_corrupt_time.csv'

if ON_COLAB:
    filename_r01_ordered = drive_path+filename_r01_ordered
    filename_r01_randomized = drive_path+filename_r01_randomized
    filename_dataport_ordered = drive_path+filename_dataport_ordered
    filename_dataport_randomized = drive_path+filename_dataport_randomized
    filename_gt_ordered = drive_path+filename_gt_ordered
    filename_gt_randomized = drive_path+filename_gt_randomized
    

DO_DECIMATION = not True
if DO_DECIMATION:
    gait_data_r01_randomized = pd.read_csv(filename_r01_randomized, nrows=10000)
    gait_data_r01_ordered = pd.read_csv(filename_r01_ordered, nrows=10000)
    gait_data_dataport_ordered = pd.read_csv(filename_dataport_ordered, nrows=10000)
    gait_data_dataport_randomized = pd.read_csv(filename_dataport_randomized, nrows=10000)
    gait_data_gt_ordered = pd.read_csv(filename_gt_ordered, nrows=10000)
    gait_data_gt_randomized = pd.read_csv(filename_gt_randomized, nrows=10000)

    
else:
    gait_data_r01_randomized = pd.read_csv(filename_r01_randomized)
    gait_data_r01_ordered = pd.read_csv(filename_r01_ordered)
    gait_data_dataport_ordered = pd.read_csv(filename_dataport_ordered)
    gait_data_dataport_randomized = pd.read_csv(filename_dataport_randomized)
    gait_data_gt_ordered = pd.read_csv(filename_gt_ordered)
    gait_data_gt_randomized = pd.read_csv(filename_gt_randomized)


# REMOVE RANDOM SUBJECTS FOR X-VALIDATION
#FROM R01, remove two: AB02 and AB06
#FROM DATAPORT, remove three: AB09, AB05, and AB10
#FROM GT, remove six, AB25, AB28, AB30, AB20, AB12, AB09
REMOVE_SUBS_XVAL = True

if REMOVE_SUBS_XVAL:
    #r01 ordered
    index_sub_remove_r01 = gait_data_r01_ordered[ (gait_data_r01_ordered['subj_id'] == 2) | (gait_data_r01_ordered['subj_id'] == 6) ].index
    gait_data_r01_ordered_val = gait_data_r01_ordered.iloc[index_sub_remove_r01]
    gait_data_r01_ordered.drop(index_sub_remove_r01 , inplace=True)
    
    #r01 randomized
    index_sub_remove_r01 = gait_data_r01_randomized[ (gait_data_r01_randomized['subj_id'] == 2) | (gait_data_r01_randomized['subj_id'] == 6) ].index
    gait_data_r01_randomized_val = gait_data_r01_randomized.iloc[index_sub_remove_r01]
    gait_data_r01_randomized.drop(index_sub_remove_r01 , inplace=True)
    
    print(gait_data_r01_ordered.head())
    print(gait_data_r01_ordered_val.head())
    
    #dataport ordered
    index_sub_remove_dataport = gait_data_dataport_ordered[ (gait_data_dataport_ordered['subj_id'] == 9) | 
                                                           (gait_data_dataport_ordered['subj_id'] == 5) |
                                                          (gait_data_dataport_ordered['subj_id'] == 10)].index

    gait_data_dataport_ordered_val = gait_data_dataport_ordered.iloc[index_sub_remove_dataport]
    gait_data_dataport_ordered.drop(index_sub_remove_dataport , inplace=True)
    
    #dataport randomized
    index_sub_remove_dataport = gait_data_dataport_randomized[ (gait_data_dataport_randomized['subj_id'] == 9) | 
                                                           (gait_data_dataport_randomized['subj_id'] == 5) |
                                                          (gait_data_dataport_randomized['subj_id'] == 10)].index

    gait_data_dataport_randomized_val = gait_data_dataport_randomized.iloc[index_sub_remove_dataport]
    gait_data_dataport_randomized.drop(index_sub_remove_dataport , inplace=True)
    
    #gt ordered
    index_sub_remove_gt = gait_data_gt_ordered[ (gait_data_gt_ordered['subj_id'] == 25) | 
                                               (gait_data_gt_ordered['subj_id'] == 28) |
                                              (gait_data_gt_ordered['subj_id'] == 30) |
                                              (gait_data_gt_ordered['subj_id'] == 20) |
                                              (gait_data_gt_ordered['subj_id'] == 12) |
                                              (gait_data_gt_ordered['subj_id'] == 9)].index
    
    gait_data_gt_ordered_val = gait_data_gt_ordered.iloc[index_sub_remove_gt]
    gait_data_gt_ordered.drop(index_sub_remove_gt , inplace=True)
    
    #gt randomized
    index_sub_remove_gt = gait_data_gt_randomized[ (gait_data_gt_randomized['subj_id'] == 25) | 
                                               (gait_data_gt_randomized['subj_id'] == 28) |
                                              (gait_data_gt_randomized['subj_id'] == 30) |
                                              (gait_data_gt_randomized['subj_id'] == 20) |
                                              (gait_data_gt_randomized['subj_id'] == 12) |
                                              (gait_data_gt_randomized['subj_id'] == 9)].index
    
    gait_data_gt_randomized_val = gait_data_gt_randomized.iloc[index_sub_remove_gt]
    gait_data_gt_randomized.drop(index_sub_remove_gt , inplace=True)
    
    #concatenate
    gait_data = pd.concat([gait_data_r01_ordered, gait_data_r01_randomized,\
                          gait_data_dataport_ordered, gait_data_dataport_randomized,\
                          gait_data_gt_ordered, gait_data_gt_randomized])
    
    gait_data_val = pd.concat([gait_data_r01_ordered_val, gait_data_r01_randomized_val,\
                              gait_data_dataport_ordered_val, gait_data_dataport_randomized_val,\
                              gait_data_gt_ordered_val, gait_data_gt_randomized_val])
    
    train_dataset = WindowedGaitDataset(gait_data=gait_data,
                                                meas_scale=meas_scale,
                                                window_size = window_size,
                                                speed_scale = speed_scale,
                                                incline_scale = incline_scale,
                                                stair_height_scale=stair_height_scale,
                                                transform=ToTensor())
    
    val_dataset = WindowedGaitDataset(gait_data=gait_data_val,
                                                meas_scale=meas_scale,
                                                window_size = window_size,
                                                speed_scale = speed_scale,
                                                incline_scale = incline_scale,
                                                stair_height_scale=stair_height_scale,
                                                transform=ToTensor())
    
    print('{:>5,} training samples'.format(len(train_dataset)))
    print('{:>5,} validation samples'.format(len(val_dataset)))


else:
    gait_data = pd.concat([gait_data_r01_ordered, gait_data_r01_randomized,\
                          gait_data_dataport_ordered, gait_data_dataport_randomized,\
                          gait_data_gt_ordered, gait_data_gt_randomized])
    # gait_data = gait_data_r01_randomized
    # gait_data = pd.concat([gait_data_r01_randomized, gait_data_dataport_randomized])
    
    dataset = WindowedGaitDataset(gait_data=gait_data,
                                            meas_scale=meas_scale,
                                            window_size = window_size,
                                            speed_scale = speed_scale,
                                            incline_scale = incline_scale,
                                            stair_height_scale=stair_height_scale,
                                            transform=ToTensor())  
    # Create a 90-10 train-validation split.

    # Calculate the number of samples to include in each set.
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size


    # Divide the dataset by randomly selecting samples.
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    print('{:>5,} training samples'.format(train_size))
    print('{:>5,} validation samples'.format(val_size))

print(val_dataset[0]['meas'][-1,:])
print(val_dataset[1]['meas'][-1,:])
print(val_dataset[2]['meas'][-1,:])






Set up Dataloaders to hold the training and validation sets

In [ ]:
#SET BATCH SIZE
BATCH_SIZE_TRAIN = 512
BATCH_SIZE_VALIDATE = 2048
NUM_WORKERS = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle=True,
            batch_size = BATCH_SIZE_TRAIN, # Trains with this batch size.
            pin_memory=True,
            num_workers=NUM_WORKERS
        )


# For validation the order doesn't matter, so we'll just read them sequentially.
#try shuffle off see if it fixes
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            shuffle=True,
            batch_size = BATCH_SIZE_VALIDATE, # Evaluate with this batch size.
            pin_memory=True,
            num_workers=NUM_WORKERS
        )

Plot some samples for sanity

In [ ]:
fig, axs = plt.subplots(3,1)

for i in range(0,80):
  y = train_dataset[i+window_size]['meas'][:,0]
  y1 = train_dataset[i+window_size]['meas'][:,4]
  x = np.array([j for j in range(len(y))])
  x = x + i

  yy = train_dataset[i+window_size]['state'][:,0]
  axs[0].plot(x,y,linewidth=3)
  axs[1].plot(x,y1,linewidth=3)
  axs[2].plot(i,yy,'o',linewidth=3)

Set up the model nickname for easy identification betweeen model types. Feel free to choose whatever you want for the nickname. Currently, apollyon-three-stairs denotes the latest gait transformer model

In [ ]:
import os


model_nickname = 'apollyon-three-stairs'

output_dir = f'../staging_area/{model_nickname}/model_save/'
if REMOVE_SUBS_XVAL:
    output_dir = f'../staging_area/{model_nickname}/model_save_xval/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

checkpoint_dir = 'checkpoints/'
if not os.path.exists(output_dir+checkpoint_dir):
    os.makedirs(output_dir+checkpoint_dir)


Set up model parameters

In [ ]:
# Model parameters
dim_val = 32 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 4 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
n_decoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 7 # The number of input variables. 1 if univariate forecasting.
enc_seq_len = 150 # length of input given to encoder. Can have any integer value.
dec_seq_len = 1 # length of input given to decoder. Can have any integer value.

dropout_encoder = 0.1
dropout_decoder = 0.1
dropout_pos_enc = 0.0
dropout_regression = 0.1
dim_feedforward_encoder = 512
dim_feedforward_decoder = 512

num_predicted_features = 5 # The number of output kinematics. 

model = GaitTransformer(
    dim_val=dim_val,
    input_size=input_size, 
    n_encoder_layers=n_encoder_layers,
    n_decoder_layers=n_decoder_layers,
    n_heads=n_heads,
    enc_seq_len=enc_seq_len,
    dropout_encoder=dropout_encoder,
    dropout_decoder=dropout_decoder,
    dropout_pos_enc=dropout_pos_enc,
    dropout_regression=dropout_regression,
    num_predicted_features=num_predicted_features,
    dim_feedforward_encoder=dim_feedforward_encoder,
    dim_feedforward_decoder=dim_feedforward_decoder,
)
    
enum_parameters(model)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

# set up the class to auto-save the best (lowest-loss) model 
best_model_name = 'ml_gait_estimator_dec_best_model.tar'
save_best_model = SaveBestModel(output_dir+best_model_name)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

#set up the optimizer. We use Adam as it's pretty good
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.998), eps=1e-9, weight_decay=1e-4)
#set up learning rate scheduler. This scheduler decreases the learning rate if error does not sufficiently decrease
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=0, threshold=0.01, verbose=True,min_lr=1e-6)

start_epoch = 0

#optionally, load a model from a checkpoint. Useful to pick back up training from a previous point
FROM_CHECKPOINT = not True
if FROM_CHECKPOINT:
    
    checkpoint = torch.load(output_dir+'ml_gait_estimator_dec_best_model.tar')
    g = checkpoint['model_state_dict']
    loss = checkpoint['loss']
    print(f'Lowest Loss: {loss}')
    save_best_model = SaveBestModel(output_dir+best_model_name, loss)
    # print(g.keys())
    model.load_state_dict(g)

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f'From epoch: {start_epoch}')


#set up the number of epochs to train the model for
epochs = 10

#set how often to save the model checkpoint
SAVE_EVERY_EPOCH_N = 1


#define loss function
lossfcn = GaitLoss(w_phase=2, w_speed=5, w_incline=10, w_stairs=2)

#Set up start of sequence token
SOS_token = 100 * torch.ones(1, 1, num_predicted_features).to(device).requires_grad_(False)

#Extract index for time steps
DT_IDX = 6

training_RMSEs = []
validation_RMSEs = []

import time
import datetime

# Measure the total training time for the whole run.
total_t0 = time.time()


for epoch_i in range(start_epoch, start_epoch+epochs):
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, start_epoch+epochs))
    print('Training...')
    
    #print learning rate, helps to keep track of how far into training we are
    lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {lr}')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()
    step_ct = 0
    for step, batch in enumerate(train_dataloader):
        step_ct += 1

        
        optimizer.zero_grad()   
        
        b_input = batch['meas'].to(device) #obtain kinematics inputs to transformer
        b_state = batch['state'].to(device) #obtain ground truth gait state
                
        tgt = SOS_token.repeat(b_state.shape[0], 1, 1) #project the SOS token to a batched input to the transformer decoder

        #extract delta time steps used to encode position of kinematics
        dts = b_input[:,:,DT_IDX]
        dts = torch.unsqueeze(dts, dim=-1)

        #RUN THE MODEL ON INPUTS! this is the forward pass
        prediction = model(b_input,tgt, dts)
        
        #compute loss
        loss = lossfcn(prediction, b_state)

        #backprop loss!
        loss.backward()

        #optimize parameters using backprop'd loss
        optimizer.step()

        #obtain value of the loss
        loss_value = loss.item()

        # Progress update every 100 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            print(loss_value)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss_value
        

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / step_ct           
    training_RMSEs.append(avg_train_loss)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    # Save to checkpoints
    if (epoch_i + 1) % SAVE_EVERY_EPOCH_N == 0:
        print('Saving model checkpoint')
        model_name = f"ml_gait_estimator_dec_checkpoint_{epoch_i + 1}.tar"
        path_name = output_dir+checkpoint_dir+model_name
        print(path_name)
        torch.save({
                    'epoch':epoch_i + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                    }, path_name)

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    step_ct = 0
    for batch in validation_dataloader:
        step_ct += 1

        b_input = batch['meas'].to(device)
        b_state = batch['state'].to(device)
                
        tgt = SOS_token.repeat(b_state.shape[0], 1, 1)
        dts = b_input[:,:,DT_IDX]
        dts = torch.unsqueeze(dts, dim=-1)

        # Don't construct the compute graph during validation forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            outputs = model(b_input, tgt, dts)

        loss = lossfcn(outputs, b_state)
        #print(loss)
        loss_value = loss.item()
        
        # Accumulate the validation loss.
        total_eval_loss += loss_value


    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / step_ct
    
    #schedule PlateauLoss on the validation loss
    scheduler.step(avg_val_loss)
    
    validation_RMSEs.append(avg_val_loss)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    
    #save best model, i.e. the model with the lowest validation loss
    save_best_model(
        avg_val_loss, epoch_i+1, model, optimizer, lossfcn
    )
    
    #print training vals
    print('Training vals')
    print(validation_RMSEs)

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

print("Saving model to %s" % output_dir)

#save model params
model_name = 'ml_gait_estimator_dec_params.pt'
torch.save(model.state_dict(), output_dir+model_name)

model_name = 'ml_gait_estimator_dec_full.pt'
torch.save(model, output_dir+model_name)

#save checkpoint
model_name = f"ml_gait_estimator_dec_checkpoint_{epoch_i + 1}.tar"
torch.save({'epoch': epoch_i + 1,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': loss.item(),
                      }, path_name)

Plot the losses over time

In [ ]:
training_RMSEs = np.array(training_RMSEs)
validation_RMSEs = np.array(validation_RMSEs)
fig, axs = plt.subplots()
axs.plot(training_RMSEs,'-',label='Train')
axs.set_ylabel('Loss (MSE)')
axs.plot(validation_RMSEs,'-',label='Val')
axs.set_xlabel('Epoch')
axs.legend()
print(np.min(validation_RMSEs))

To show model performance, run the model on the entirety of the val data, and compute individual gait state RMSEs

In [ ]:
#Test model
test_dataset = val_dataset

BATCH_SIZE = 1024*4
prediction_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False,num_workers=8)
# Prediction on test set

print('Predicting labels for {:,} test points...'.format(len(test_dataset)))

# Model parameters
dim_val = 32 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 4 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
n_decoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 7 # The number of input variables. 1 if univariate forecasting.
enc_seq_len = 150 # length of input given to encoder. Can have any integer value.
dec_seq_len = 1 # length of input given to decoder. Can have any integer value.

dropout_encoder = 0.1
dropout_decoder = 0.1
dropout_pos_enc = 0.0
dropout_regression = 0.1
dim_feedforward_encoder = 512
dim_feedforward_decoder = 512

num_predicted_features = 5 # The number of output variables. 

best_model = GaitTransformer(
    dim_val=dim_val,
    input_size=input_size, 
    n_encoder_layers=n_encoder_layers,
    n_decoder_layers=n_decoder_layers,
    n_heads=n_heads,
    enc_seq_len=enc_seq_len,
    dropout_encoder=dropout_encoder,
    dropout_decoder=dropout_decoder,
    dropout_pos_enc=dropout_pos_enc,
    dropout_regression=dropout_regression,
    num_predicted_features=num_predicted_features,
    dim_feedforward_encoder=dim_feedforward_encoder,
    dim_feedforward_decoder=dim_feedforward_decoder,
)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")


best_model.to(device)

model_nickname = 'apollyon-three-stairs'

model_dir = f'../staging_area/{model_nickname}/model_save/'
if REMOVE_SUBS_XVAL:
    model_dir = f'../staging_area/{model_nickname}/model_save_xval/'
    

checkpoint = torch.load(model_dir+'ml_gait_estimator_dec_best_model.tar')
g = checkpoint['model_state_dict']
loss = checkpoint['loss']
print(f'Lowest Loss: {loss}')
best_model.load_state_dict(g)


epoch = checkpoint['epoch']

# Put model in evaluation mode
best_model.eval()

# Tracking variables 
predictions_raw, true_labels_raw = [], []

#Set up start of sequence token
SOS_token = 100 * torch.ones(1, 1, num_predicted_features).to(device).requires_grad_(False)

#Extract index for time steps
DT_IDX = 6


# Predict 
print(len(prediction_dataloader))
total_t0 = time.time()

for batch in prediction_dataloader:

    b_input = batch['meas'].to(device)
    b_state = batch['state'].to(device)
    
    tgt = SOS_token.repeat(b_state.shape[0], 1, 1)
    dts = b_input[:,:,DT_IDX]
    dts = torch.unsqueeze(dts, dim=-1)


    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = best_model(b_input,tgt, dts)


    # Move logits and labels to CPU
    outputs = outputs.detach().to('cpu').numpy()
    b_state = b_state.to('cpu').numpy()
  
    # Store predictions and true labels
    outputs = np.squeeze(outputs, axis=1)
    b_state = np.squeeze(b_state, axis=1)
    
    # print(outputs.shape)
    
    #unscale
    outputs = unscale_gait_state(outputs, speed_scale, incline_scale, stair_height_scale)
    b_state = unscale_gait_state(b_state, speed_scale, incline_scale, stair_height_scale)

    # Store predictions and true labels
    predictions_raw.extend(outputs.tolist())
    true_labels_raw.extend(b_state.tolist())

print('    DONE PREDICTING')
print("Total predicting took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


Compute gait state RMSEs and plot the predicted vs actual gait states

In [ ]:
predictions = np.array(predictions_raw)
true_labels = np.array(true_labels_raw)

#round stairs to three locomotion modes
true_labels[true_labels[:,3] < -0.5,3] = -1
true_labels[np.logical_and(true_labels[:,3] >= -0.5, true_labels[:,3] <= 0.5),3] = 0
true_labels[true_labels[:,3] > 0.5,3] = 1

predictions[predictions[:,3] < -0.5,3] = -1
predictions[np.logical_and(predictions[:,3] >= -0.5, predictions[:,3] <= 0.5),3] = 0
predictions[predictions[:,3] > 0.5,3] = 1


phase_losses = np.sqrt(phase_dist(predictions[:,0], true_labels[:,0])**2)
speed_losses = np.sqrt((predictions[:,1] - true_labels[:,1])**2)
incline_losses = np.sqrt((predictions[:,2] - true_labels[:,2])**2)

stair_height_accuracy = np.sum(true_labels[:,3] == predictions[:,3])/len(true_labels[:,3])
stair_height_accuracy_ascent = np.sum(true_labels[true_labels[:,3] == 1,3] == predictions[true_labels[:,3] == 1,3])/len(true_labels[true_labels[:,3] == 1,3])
stair_height_accuracy_descent = np.sum(true_labels[true_labels[:,3] == -1,3] == predictions[true_labels[:,3] == -1,3])/len(true_labels[true_labels[:,3] == -1,3])


print(predictions.shape)

print("="*30)
print(f'Phase Losses: {np.mean(phase_losses):.3f} +- {np.std(phase_losses):.3f}')
print(f'Speed Losses: {np.mean(speed_losses):.3f} +- {np.std(speed_losses):.3f}')
print(f'Incline Losses: {np.mean(incline_losses):.3f} +- {np.std(incline_losses):.3f}')
# print(f'Stair Height Losses: {np.mean(stair_height_losses):.3f} +- {np.std(stair_height_losses):.3f}')
# print(f'Stair Height Losses On Stairs: {np.mean(stair_height_losses_on_stairs):.3f} +- {np.std(stair_height_losses_on_stairs):.3f}')

print(f'Is Stairs Accuracy: {stair_height_accuracy:.3f}')
print(f'Is Stairs Accuracy, Ascent: {stair_height_accuracy_ascent:.3f}')
print(f'Is Stairs Accuracy, Descent: {stair_height_accuracy_descent:.3f}')


print(predictions.shape)



fig, axs = plt.subplots(4,1,figsize=(20,8),sharex=True)
axs[0].plot(predictions[:,0],'r',label='predict')
axs[0].plot(true_labels[:,0],'b',label='actual')
axs[0].legend()
# axs[0].set_xlim([9000,20000])
# axs[0].set_xlim([0.5e6,0.5e6+5000])
# axs[0].set_xlim([80,250000]) #ordered r01
# axs[0].set_xlim([1.47e5,1.75e5]) #ordered r01 stairs

# axs[0].set_xlim([80+1000,0.05e6+1000])
# axs[0].set_xlim([0.3622e6,0.3725e6]) #r01 randomized mix of stairs and inclines
# axs[0].set_xlim([0.25e6,0.26e6])
# axs[0].set_xlim([2e6,2.02e6]) #dataport mix of ramps
# axs[0].set_xlim([0.15e6,0.16e6])
axs[0].set_xlim([3.175e6,3.19e6]) #ordered gt
# axs[0].set_xlim([3.5e6,3.53e6]) #randomized gt

axs[0].set_ylabel('Phase')

axs[1].plot(predictions[:,1],'r',label='predict')
axs[1].plot(true_labels[:,1],'b',label='actual')
axs[1].set_ylabel('Speed (m/s)')
axs[1].set_ylim([-1.25,2])


axs[2].plot(predictions[:,2],'r',label='predict')
axs[2].plot(true_labels[:,2],'b',label='actual')
axs[2].set_ylabel('Incline (deg)')
# axs[2].set_ylim([-15,15])

axs[3].plot(predictions[:,3],'r.',label='predict')
axs[3].plot(true_labels[:,3],'b',label='actual')
axs[3].set_ylabel('Is Stairs')
# axs[3].set_ylim([-0.2,1.2])


For use in Vertex AI: copy the models from the preset staging_area folder (which isn't tracked via GitHub) to the tracked full_models folder to save the model

In [ ]:
!gsutil cp -r ../staging_area/apollyon-three-stairs/ ../full_models/
!zip -r ../full_models/apollyon-three-stairs.zip ../full_models/apollyon-three-stairs

For use in Vertex AI: copy the models from the tracked full_models folder to a dedicated bucket

In [ ]:
!gsutil cp -r ../full_models/apollyon-three-stairs/ gs://ml_gait_estimation/full_models/
!gsutil cp -r ../full_models/apollyon-three-stairs.zip gs://ml_gait_estimation/full_models/
